In [6]:
import torch

ckpt = torch.load("/media/yuganlab/blackstone/clip_classifier_epoch2.pt", map_location="cpu")
print("\n".join(k for k in ckpt.keys()))

/tmp/ipykernel_692984/996909386.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("/media/yuganlab/blackstone/clip_classifier_epoch2.pt", map_location="c

clip_model.positional_embedding
clip_model.text_projection
clip_model.logit_scale
clip_model.visual.class_embedding
clip_model.visual.positional_embedding
clip_model.visual.proj
clip_model.visual.conv1.weight
clip_model.visual.ln_pre.weight
clip_model.visual.ln_pre.bias
clip_model.visual.transformer.resblocks.0.ln_1.weight
clip_model.visual.transformer.resblocks.0.ln_1.bias
clip_model.visual.transformer.resblocks.0.attn.in_proj_weight
clip_model.visual.transformer.resblocks.0.attn.in_proj_bias
clip_model.visual.transformer.resblocks.0.attn.out_proj.weight
clip_model.visual.transformer.resblocks.0.attn.out_proj.bias
clip_model.visual.transformer.resblocks.0.ln_2.weight
clip_model.visual.transformer.resblocks.0.ln_2.bias
clip_model.visual.transformer.resblocks.0.mlp.c_fc.weight
clip_model.visual.transformer.resblocks.0.mlp.c_fc.bias
clip_model.visual.transformer.resblocks.0.mlp.c_proj.weight
clip_model.visual.transformer.resblocks.0.mlp.c_proj.bias
clip_model.visual.transformer.resblocks

In [1]:
import torch
from diffusers import KandinskyV22PriorPipeline, AutoPipelineForImage2Image, AutoPipelineForInpainting
from transformers import CLIPTextModelWithProjection, CLIPTokenizer, CLIPVisionModelWithProjection
from diffusers.utils import load_image, make_image_grid

ckpt = torch.load("/media/yuganlab/blackstone/clip_classifier_epoch3.pt", map_location="cpu")

# 2) Pull out the 'backbone' sub‑dict and rename to 'text_model'
# vis_sd = {}
# for k,v in ckpt.items():
#     # only take the visual‐backbone keys if your backbone was actually the text encoder;
#     # adjust the prefix if you used a different attribute name
#     if k.startswith("backbone."):
#         new_k = k.replace("backbone.", "vision_model.")
#         vis_sd[new_k] = v


/media/yuganlab/blackstone/xinlong/.conda/envs/ss2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_1229741/918833328.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend

In [8]:


device = torch.device("cuda:1")

pipe_prior = KandinskyV22PriorPipeline.from_pretrained(
    "kandinsky-community/kandinsky-2-2-prior", torch_dtype=torch.float16, use_safetensors=True
).to(device)

vision_cfg     = pipe_prior.image_encoder.config
ft_vis_encoder = CLIPVisionModelWithProjection(vision_cfg).to(device)

text_cfg     = pipe_prior.text_encoder.config
ft_text_encoder = CLIPTextModelWithProjection(text_cfg).to(device)

ft_vis_encoder.load_state_dict(ckpt, strict=False)
ft_text_encoder.load_state_dict(ckpt, strict=False)


# ── E) Swap it into the pipeline ───────────────────────────────────────────

pipe_prior.image_encoder = ft_vis_encoder
pipe_prior.text_encoder = ft_text_encoder

pipe_prior.text_encoder    = pipe_prior.text_encoder.to(device).float()
pipe_prior.image_encoder = pipe_prior.image_encoder.to(device).float()
pipe_prior.prior           = pipe_prior.prior.to(device).float()


pipe = AutoPipelineForInpainting.from_pretrained(
    "kandinsky-community/kandinsky-2-2-decoder-inpaint",
    prior=pipe_prior.prior,

    # 2) text side
    tokenizer=pipe_prior.tokenizer,
    text_encoder=pipe_prior.text_encoder,

    # 3) image‑prompt side
    clip_image_encoder=pipe_prior.image_encoder,
    clip_image_processor=pipe_prior.image_processor,

    # misc
    torch_dtype=torch.float32,
    use_safetensors=True
).to(device)


Loading pipeline components...: 100%|██████████| 6/6 [00:01<00:00,  4.30it/s]
Keyword arguments {'prior': PriorTransformer(
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=2048, out_features=2048, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=2048, out_features=2048, bias=True)
  )
  (proj_in): Linear(in_features=1280, out_features=2048, bias=True)
  (embedding_proj): Linear(in_features=1280, out_features=2048, bias=True)
  (encoder_hidden_states_proj): Linear(in_features=1280, out_features=2048, bias=True)
  (transformer_blocks): ModuleList(
    (0-19): 20 x BasicTransformerBlock(
      (norm1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (attn1): Attention(
        (to_q): Linear(in_features=2048, out_features=2048, bias=True)
        (to_k): Linear(in_features=2048, out_features=2048, bias=True)
        (to_v): Linear(in_features=2048, out_features=2048, bias=True)
        (to_out): ModuleList(
 

In [ ]:
from PIL import Image

image_path = "/media/yuganlab/blackstone/HZL/Retinal_seg/Retinal-vessel-segmentation/data/test/image/01_test_0.png"
original_image = Image.open(image_path).convert("RGB")

mask_path = "/media/yuganlab/blackstone/HZL/Retinal_seg/Retinal-vessel-segmentation/data/test/mask/01_test_0.png"
mask_image = Image.open(mask_path).convert("L")
prompt = "a binary mask of an eyeball vascular strucutre with increased tortuosity, vascular strucutre only"
out = pipe(
    prompt= "increase vessel tortuosity to high tortuosity level, keep binary mask of vascular structure",  # what you want
    image=original_image,               # your starting image
    mask_image=mask_image,                    # None → pure img2img       # how strongly to follow your prompt
    num_inference_steps=150,
    generator=torch.Generator(device=device).manual_seed(1234),
).images[0]

out.save("clip_gen.png")

100%|██████████| 150/150 [00:11<00:00, 12.97it/s]


: 